PyGaze Analysis

In [132]:
# We load the original results from the file data/4/processed-data/fixationsAll.csv

import pandas as pd

# We load the original results from the file data/4/processed-data/fixationsAll.csv
participants_metadata = pd.read_csv('data/emip_dataset/emip_metadata.csv')

# We load the PyGazeAnalyzer results from the file results/pygaze_fixations_4.csv
pygaze_fixations = pd.read_csv('results/pygaze_fixations_emip_dataset.csv')


# We combine the two dataframes into one by combining the columns where the values of Participant are the same
results_pygaze = pd.merge(participants_metadata, pygaze_fixations, how='inner', left_on=['id'], right_on=['Participant'])
results_pygaze


,id,age,gender,mother_tongue,english_level,visual_aid,makeup,experiment_language,expertise_experiment_language,time_experiment_language,...,stimulus_vehicle,mother_tongue_original,time_experiment_language_original,time_programming_original,other_languages_original,Participant,Task,Fixation Count,Total Fixation Duration [ms],Average Fixation Duration [ms]
0,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,vehicle_java.jpg,247,33581.623,135.957988
1,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_vehicle.jpg,114,17408.333,152.704675
2,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,rectangle_java.jpg,67,6502.875,97.057836
3,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_rectangle.jpg,128,20457.473,159.824008
4,2,24,male,Slovak,medium,glasses,no,Java,medium,2.0,...,vehicle_java2.jpg,Slovak,2,5,"c++(low), php(medium), delphi(low)",2,rectangle_java2.jpg,131,12446.719,95.013122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,215,22,male,Slovak,medium,no,no,Java,medium,1.0,...,vehicle_java2.jpg,slovak,1,2,C (medium) c# (medium) Java (medium),215,mupliple_choice_rectangle.jpg,113,18223.012,161.265593
860,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,vehicle_java2.jpg,296,29761.160,100.544459
861,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,mupliple_choice_vehicle.jpg,72,8699.441,120.825569
862,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,rectangle_java2.jpg,150,13818.919,92.126127


In [133]:
# We load the original results from the file data/4/processed-data/fixationsAll.csv

import pandas as pd

# We load the original results from the file data/4/processed-data/fixationsAll.csv
participants_metadata = pd.read_csv('data/emip_dataset/emip_metadata.csv')

# We load the PyGazeAnalyzer results from the file results/pygaze_fixations_4.csv
pygaze_saccades = pd.read_csv('results/pygaze_saccades_emip_dataset.csv')


# We combine the two dataframes into one by combining the columns where the values of Participant are the same
results_pygaze_saccades = pd.merge(participants_metadata, pygaze_saccades, how='inner', left_on=['id'], right_on=['Participant'])
results_pygaze_saccades


,id,age,gender,mother_tongue,english_level,visual_aid,makeup,experiment_language,expertise_experiment_language,time_experiment_language,...,mother_tongue_original,time_experiment_language_original,time_programming_original,other_languages_original,Participant,Task,Saccade Count,Total Saccade Duration [ms],Average Saccade Duration [ms],Average Saccade Distance [px]
0,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,vehicle_java.jpg,76,74215.197,976.515750,134.748075
1,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_vehicle.jpg,66,38972.494,590.492333,146.738515
2,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,rectangle_java.jpg,17,27547.014,1620.412588,124.579446
3,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_rectangle.jpg,60,35512.116,591.868600,97.634411
4,2,24,male,Slovak,medium,glasses,no,Java,medium,2.0,...,Slovak,2,5,"c++(low), php(medium), delphi(low)",2,rectangle_java2.jpg,5,16226.784,3245.356800,144.592684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,215,22,male,Slovak,medium,no,no,Java,medium,1.0,...,slovak,1,2,C (medium) c# (medium) Java (medium),215,mupliple_choice_rectangle.jpg,43,21678.556,504.152465,115.495182
860,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,Thai,6 weeks,6 weeks,Low,216,vehicle_java2.jpg,33,68278.123,2069.034030,172.396854
861,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,Thai,6 weeks,6 weeks,Low,216,mupliple_choice_vehicle.jpg,12,16626.615,1385.551250,159.478738
862,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,Thai,6 weeks,6 weeks,Low,216,rectangle_java2.jpg,13,33212.721,2554.824692,107.542449


In [6]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu

def show_results1(results):
	# We calculate the mean and the standard deviation of the fixation count for each CategoryName
		# We first calculate the mean for each participant
	results_mean_per_subject = results.groupby(['CategoryName', 'Subject']).mean(numeric_only=True)[["Fixation Count"]]
	results_std_per_subject = results.groupby(['CategoryName', 'Subject']).std(numeric_only=True)[["Fixation Count"]]

	print(results_mean_per_subject)

	results_mean = results_mean_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count"]]
	results_std = results_std_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count"]]

	print(results_mean)


	#results_mean = results.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	#results_std = results.groupby(['CategoryName']).std(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	# Apply Pairwise Comparisons Using Non-Parametric Wilcox Tests (α = 0.05)
	# TODO: check if there is a way to use wilcox 

	stat_list_vs_mental_fix_count, p_list_vs_mental_fix_count = wilcoxon(results_mean_per_subject['Fixation Count'], results_std_per_subject['Fixation Count'])

	# Use mannwhitneyu for the other comparison
	stat_tree_vs_mental_fix_count, p_tree_vs_mental_fix_count = mannwhitneyu(results[results['CategoryName'] == "Tree"]['Fixation Count'], results[results['CategoryName'] == "Mental"]['Fixation Count'])

	stat_list_vs_mental_fix_time, p_list_vs_mental_fix_time = mannwhitneyu(results[results['CategoryName'] == "List"]['Total Fixation Duration [ms]'], results[results['CategoryName'] == "Mental"]['Total Fixation Duration [ms]'])

	# Use mannwhitneyu for the other comparison
	stat_tree_vs_mental_fix_time, p_tree_vs_mental_fix_time = mannwhitneyu(results[results['CategoryName'] == "Tree"]['Total Fixation Duration [ms]'], results[results['CategoryName'] == "Mental"]['Total Fixation Duration [ms]'])

	# We create a table to show the results

	# We divide the fixation time by 1000 to get the fixation time in seconds
	results_mean['Total Fixation Duration [ms]'] = results_mean['Total Fixation Duration [ms]']/1000
	results_std['Total Fixation Duration [ms]'] = results_std['Total Fixation Duration [ms]']/1000

	data = [

		[
			'Fixation Time (s)', 
			f"{'%.2f' % results_mean.loc['List']['Total Fixation Duration [ms]']}({'%.4f' % results_std.loc['List']['Total Fixation Duration [ms]']})",
			f"{'%.4f' % results_mean.loc['Mental']['Total Fixation Duration [ms]']}({'%.4f' % results_std.loc['Mental']['Total Fixation Duration [ms]']})",
			f"{'%.4f' % results_mean.loc['Tree']['Total Fixation Duration [ms]']}({'%.4f' % results_std.loc['Tree']['Total Fixation Duration [ms]']})",
			'%.4f' % p_list_vs_mental_fix_time,
			'%.4f' % p_tree_vs_mental_fix_time
		],
		[
			'Fixation Count', 
			f"{'%.0f' % results_mean.loc['List']['Fixation Count']} ({'%.0f' % results_std.loc['List']['Fixation Count']})", 
			f"{'%.0f' % results_mean.loc['Mental']['Fixation Count']} ({'%.0f' % results_std.loc['Mental']['Fixation Count']})", 
			f"{'%.0f' % results_mean.loc['Tree']['Fixation Count']} ({'%.0f' % results_std.loc['Tree']['Fixation Count']})", 
			'%.4f' % p_list_vs_mental_fix_count, 
			'%.4f' % p_tree_vs_mental_fix_count
		]

		
	]
	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'List', 'Mental', 'Tree', 'List vs Mental p', 'Tree vs Mental p']
	return data_df


In [7]:

from pandas import Interval

def show_results2(results):

	results_std = results[["gender", "age", "Fixation Count", "Average Fixation Duration [ms]"]].std(numeric_only=True)

	# Group the results by the different values of gender
	results_sex = results.groupby("gender")[["Fixation Count", "Average Fixation Duration [ms]"]]
	results_sex_mean = results_sex.mean(numeric_only=True)
	results_sex_std = results_sex.std(numeric_only=True)

	# Group the results by ages between 18-22 and 23-27
	results_age = results.groupby(pd.cut(results["age"], [18, 22, 27]))[["Fixation Count", "Average Fixation Duration [ms]"]]
	results_age_mean = results_age.mean(numeric_only=True)
	results_age_std = results_age.std(numeric_only=True)

	# Perform non-parametric Wilcoxon tests (α =0.05) 
	stat_gender_fix_count, p_gender_fix_count = mannwhitneyu(results[results['gender'] == "male"]['Fixation Count'], results[results['gender'] == "female"]['Fixation Count'])
	stat_gender_fix_time, p_gender_fix_time = mannwhitneyu(results[results['gender'] == "male"]['Average Fixation Duration [ms]'], results[results['gender'] == "female"]['Average Fixation Duration [ms]'])

	stat_age_fix_count, p_age_fix_count = mannwhitneyu(results[results['age'] <= 22]['Fixation Count'], results[results['age'] > 22]['Fixation Count'])
	stat_age_fix_time, p_age_fix_time = mannwhitneyu(results[results['age'] <= 22]['Average Fixation Duration [ms]'], results[results['age'] > 22]['Average Fixation Duration [ms]'])

	# We create a table to show the results
	data = [
		[
			'Avg. Fix. Duration (ms)', 
			f"{'%.4f' % results_sex_mean.loc['male']['Average Fixation Duration [ms]']}({'%.4f' % results_sex_std.loc['male']['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_sex_mean.loc['female']['Average Fixation Duration [ms]']}({'%.2f' % results_sex_std.loc['female']['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_age_mean.loc[Interval(18, 22)]['Average Fixation Duration [ms]']}({'%.2f' % results_age_std.loc[Interval(18, 22)]['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_age_mean.loc[Interval(22, 27)]['Average Fixation Duration [ms]']}({'%.2f' % results_age_std.loc[Interval(22, 27)]['Average Fixation Duration [ms]']})",
			'%.2f' % p_gender_fix_time,
			'%.2f' % p_age_fix_time
		],
		[
			'Fixation Count', 
			f"{'%.0f' % results_sex_mean.loc['male']['Fixation Count']} ({'%.0f' % results_sex_std.loc['male']['Fixation Count']})", 
			f"{'%.0f' % results_sex_mean.loc['female']['Fixation Count']} ({'%.0f' % results_sex_std.loc['female']['Fixation Count']})", 
			f"{'%.0f' % results_age_mean.loc[Interval(18, 22)]['Fixation Count']} ({'%.0f' % results_age_std.loc[Interval(18, 22)]['Fixation Count']})", 
			f"{'%.0f' % results_age_mean.loc[Interval(22, 27)]['Fixation Count']} ({'%.0f' % results_age_std.loc[Interval(22, 27)]['Fixation Count']})", 
			'%.2f' % p_gender_fix_count, 
			'%.2f' % p_age_fix_count
		]
	]

	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'Men', 'Women', '18-22', '23-27', 'Gender p', 'Age p']
	return data_df



PyGaze vs Ogama

In [8]:
# Function to make the file compatible with Ogama
import pandas as pd

# We load the original data from the file data/4/formatted-raw-data/151/ogama.txt
data = pd.read_csv('data/4/formatted-raw-data/153/ogama.txt', sep=',', skiprows=0, encoding='utf-16')

# Change the values of the ImageName column to remove the last 4 characters
data[' ImageName'] = data[' ImageName'].str[:-4]

# We remove all rows where Included? is "N"
data = data[data[' Included?'] == "Y"]

# Save it again as ogama2.csv
data.to_csv('data/4/formatted-raw-data/153/changed.csv', index=False)

Import data automatically into Ogama by manipulating the database 

In [4]:
import pandas as pd
import os
import sqlite3
import numpy as np
# Need to register numpy types to be able to save them in the database, as SQL only supports int8
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

# Path to database
database_path = 'G:\\Dokumente\\OgamaExperiments\\ExperimentEMIP\\Database\\ExperimentEMIP.db'

trial_to_num = {
		'mupliple_choice_rectangle.jpg': '1',
		'mupliple_choice_vehicle.jpg': '2',
		'rectangle_java.jpg': '3',
		'rectangle_java2.jpg': '4',
		'rectangle_python.jpg': '5',
		'rectangle_scala.jpg': '6',
		'vehicle_java.jpg': '7',
		'vehicle_java2.jpg': '8',
		'vehicle_python.jpg': '9',
		'vehicle_scala.jpg': '10',
	}

def add_to_sql(c, df_msg, subject, last_task_name):

	#df_msg = df_msg[(df_msg['L Validity'] == 1) & (df_msg['R Validity'] == 1)].copy()
	df_msg = df_msg[(df_msg['R Validity'] == 1)].copy()

	if df_msg.size == 0:
		print("df_msg is empty", last_task_name)
		return

	# Define parameters for fixation detection
	x_left = df_msg.loc[:,('L Raw X [px]')].astype(float)
	#x_right = df_msg.loc[:,('R Raw X [px]')]
	x_right = df_msg.loc[:,('R POR X [px]')]
	#df_msg.loc[:,('X')] = (x_left + x_right) / 2
	df_msg.loc[:,('X')] = x_right

	y_left = df_msg['L Raw Y [px]']
	#y_right = df_msg['R Raw Y [px]']
	y_right = df_msg.loc[:,('R POR Y [px]')]
	#df_msg.loc[:,('Y')] = (y_left + y_right) / 2
	df_msg.loc[:,('Y')] = y_right
	

	# Add a table for the subject called Subject + 'RawData'
	c.execute("CREATE TABLE IF NOT EXISTS ["+ subject +"Rawdata] ([ID] integer PRIMARY KEY AUTOINCREMENT NOT NULL,[SubjectName] varchar(50) NOT NULL COLLATE NOCASE, [TrialSequence] integer NOT NULL, [Time] integer NOT NULL, [PupilDiaX] float, [PupilDiaY] float, [GazePosX] float, [GazePosY] float, [MousePosX] float, [MousePosY] float, [EventID] integer)")

	# Add data in the following scheme:
	# SubjectName = subject
	# TrialSequence = ' ImageName'
	# Time = ' StartTime'
	# GazePosX = ' X'
	# GazePosY = ' Y'

	# For each row in the dataframe
	for index, row in df_msg.iterrows():
		# Insert the data into the table
		c.execute("INSERT INTO ["+ subject +"Rawdata] (SubjectName, TrialSequence, Time, GazePosX, GazePosY) VALUES (?, ?, ?, ?, ?)", (subject, trial_to_num[last_task_name], row['Time'], row['X'], row['Y']))

	trialstarttime = df_msg['Time'].min()
	# Endtime - Starttime + 4 ms for the duration of the last measuremnt
	trialduration = df_msg['Time'].max() - df_msg['Time'].min() + 4
	if np.isnan(trialduration):
		print("trialduration is nan", trialstarttime, df_msg['Time'].max(), df_msg['Time'].min())
		print(df_msg)

	trialduration = int(trialduration)
	c.execute("INSERT INTO Trials (SubjectName, TrialID, TrialName, TrialSequence, Category,  TrialStartTime, Duration) VALUES (?, ?, ?, ?, ?, ?, ?)", (subject, trial_to_num[last_task_name], last_task_name, trial_to_num[last_task_name], "",trialstarttime, trialduration))

def import_data_into_ogama(path, subject):

	print("prepare_tobii_data", path)

	# Load the Tobii eye tracker data into a Pandas DataFrame and skip lines that start with ## as they are comments

	df = None
	possible_skipped_rows = [37, 32, 41, 45]
	counter = 0
	while df is None:
		try:
			df = pd.read_csv(path, delimiter='\t', low_memory=False, on_bad_lines='skip', skiprows=possible_skipped_rows[counter])
			df = df[['Time', 'Type', 'L Raw X [px]', 'L Raw Y [px]', 'R Raw X [px]', 'R Raw Y [px]', 'L Validity', 'R Validity', 'R POR X [px]', 'R POR Y [px]']]
		except KeyError as e:
			counter = counter + 1
			df = None
			if counter >= len(possible_skipped_rows):
				print("no possible skipped rows worked", e)
				break
		else:
			#print("possible skipped rows worked", possible_skipped_rows[counter])
			break

	df['Type'] = df['Type'].astype(str)
	df = df.fillna(0.0)

	# Normalize time
	df['Time'] = df['Time'] - df['Time'].min()
	# Time conversion from microseconds to milliseconds
	df['Time'] = df['Time'] / 1000
	# Time as integer
	df['Time'] = df['Time'].astype('int32')

	# Get the row numbers where Type is 'MSG'
	msg_rows = df[df['Type'] == 'MSG'].index

	# For each msg_row, split the df into two dataframes, one before the msg_row and one after
	# Run the analysis for each of those split dataframes as they represent different tasks
	df_after_msg = df
	last_task_name = None
	last_task_row_number = 0
	removed_task_names = {
		'instruction_calibration.jpg': True,
		'instruction_comprehension.jpg': True,
	}


	# Connect to database
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	for msg_row in msg_rows:
		current_task_name = df['L Raw X [px]'][msg_row].split('Message: ')[1]
		if '.jpg' in current_task_name:			
			if last_task_name is not None and last_task_name not in removed_task_names:
				
				df_msg = df[last_task_row_number:msg_row]
				if df_msg.size > 0:
					add_to_sql(c, df_msg, subject, last_task_name)
				else:
					print("df_msg is empty", last_task_name, last_task_row_number, msg_row)
				

			last_task_name = current_task_name
			last_task_row_number = msg_row

	df_msg = df[last_task_row_number:]
	add_to_sql(c, df_msg, subject, last_task_name)
	

	# Add subject to the Subjects table
	c.execute("INSERT INTO Subjects (SubjectName) VALUES (?)", (subject,))

	# Save the database
	conn.commit()

def drop_all_subject_tables():
	# Connect to database
	import sqlite3
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	for i in range(1, 217):
		# Drop the table named S + str(i) + "RawData"
		c.execute("DROP TABLE IF EXISTS S" + str(i) + "Rawdata")

	# Delete entries from Trials
	c.execute("DELETE FROM Trials")

	# Delete entries from Subjects
	c.execute("DELETE FROM Subjects")

	# Save the database
	conn.commit()

drop_all_subject_tables()

#for i in range(22, 23):
for i in range(1, 217):
	# If the file i+'_rawdata.tsv' exists
	filepath = 'data/emip_dataset/rawdata/'+ str(i) + '_rawdata.tsv'
	if os.path.isfile(filepath):
		# Import the data into the database
		print("import_data_into_ogama", filepath, 'S' + str(i))
		import_data_into_ogama(filepath, 'S' + str(i))

import_data_into_ogama data/emip_dataset/rawdata/1_rawdata.tsv S1
prepare_tobii_data data/emip_dataset/rawdata/1_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/2_rawdata.tsv S2
prepare_tobii_data data/emip_dataset/rawdata/2_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/3_rawdata.tsv S3
prepare_tobii_data data/emip_dataset/rawdata/3_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/4_rawdata.tsv S4
prepare_tobii_data data/emip_dataset/rawdata/4_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/5_rawdata.tsv S5
prepare_tobii_data data/emip_dataset/rawdata/5_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/6_rawdata.tsv S6
prepare_tobii_data data/emip_dataset/rawdata/6_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/7_rawdata.tsv S7
prepare_tobii_data data/emip_dataset/rawdata/7_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/8_rawdata.tsv S8
prepare_tobii_data data/emip_dataset/rawdata/8_rawdata.tsv


Now enter Ogama, load the experiment that is using this database and run the fixation algorithm.

Import the fixations from the database and calculate our results

In [157]:
import pandas as pd
import os

# Path to database
database_path = 'G:\\Dokumente\\OgamaExperiments\\ExperimentEMIP\\Database\\ExperimentEMIP.db'

total_results = pd.DataFrame()

def calculate_results_for_subject(subject):
	# Connect to database
	import sqlite3
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	# Export all fixations for the subject
	c.execute("SELECT * FROM GazeFixations WHERE SubjectName = ?", (subject,))
	fixations = c.fetchall()


	if len(fixations) == 0:
		# Create an empty dataframe with the columns ID, SubjectName, TrialID, TrialSequence, CountInTrial, StartTime, Length, PosX, PosY
		fixations_df = pd.DataFrame(columns=['ID', 'SubjectName', 'TrialID', 'TrialSequence', 'CountInTrial', 'StartTime', 'Length', 'PosX', 'PosY'])
	else:
		# Save it into a dataframe
		fixations_df = pd.DataFrame(fixations)

		# Set the column names of the dataframe
		fixations_df.columns = ['ID', 'SubjectName', 'TrialID', 'TrialSequence', 'CountInTrial', 'StartTime', 'Length', 'PosX', 'PosY']

	# For each TrialID, we calculate the total fixation count, fixation duration and average fixation duration
	results = pd.DataFrame()
	results['Total Fixation Count'] = fixations_df.groupby('TrialID')['ID'].count()
	results['Total Fixation Duration'] = fixations_df.groupby('TrialID')['Length'].sum()
	results['Average Fixation Duration'] = fixations_df.groupby('TrialID')['Length'].mean()
	results['Subject'] = subject


	return results

	


for i in range(1, 217):
	# If the folder i exists
	path = 'data/emip_dataset/rawdata/' + str(i) + '_rawdata.tsv'
	if os.path.isfile(path):
		# Append to the total_results dataframe
		total_results = pd.concat([total_results, calculate_results_for_subject('S' + str(i))])


total_results['Subject'] = total_results['Subject'].str[1:].astype(int)
total_results = total_results.reset_index()

num_to_trial = {
    '1': 'mupliple_choice_rectangle.jpg',
    '2': 'mupliple_choice_vehicle.jpg',
    '3': 'rectangle_java.jpg',
    '4': 'rectangle_java2.jpg',
    '5': 'rectangle_python.jpg',
    '6': 'rectangle_scala.jpg',
    '7': 'vehicle_java.jpg',
    '8': 'vehicle_java2.jpg',
    '9': 'vehicle_python.jpg',
    '10': 'vehicle_scala.jpg',
}

# Change the TrialID to the corresponding task name
total_results['TrialID'] = total_results['TrialID'].astype(str)
total_results['TrialID'] = total_results['TrialID'].map(num_to_trial)


# We load the original results from the file data/4/processed-data/fixationsAll.csv
participants_metadata = pd.read_csv('data/emip_dataset/emip_metadata.csv')


# We combine the two dataframes into one by combining the columns where the values of Participant and Task are the same
results_ogama = pd.merge(participants_metadata, total_results, how='inner', left_on=['id'], right_on=['Subject'])
# Rename the columns Fixation Count, Total Fixation Duration and Average Fixation Duration
results_ogama = results_ogama.rename(columns={
	'Total Fixation Count': 'Fixation Count', 
	'Total Fixation Duration': 'Total Fixation Duration [ms]', 
	'Average Fixation Duration': 'Average Fixation Duration [ms]',
	'TrialID': 'Task'
})

results_ogama


,id,age,gender,mother_tongue,english_level,visual_aid,makeup,experiment_language,expertise_experiment_language,time_experiment_language,...,stimulus_vehicle,mother_tongue_original,time_experiment_language_original,time_programming_original,other_languages_original,Task,Fixation Count,Total Fixation Duration [ms],Average Fixation Duration [ms],Subject
0,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,mupliple_choice_rectangle.jpg,94,25315,269.308511,1
1,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,mupliple_choice_vehicle.jpg,87,20467,235.252874,1
2,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,rectangle_java.jpg,49,9865,201.326531,1
3,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,vehicle_java.jpg,194,43131,222.324742,1
4,2,24,male,Slovak,medium,glasses,no,Java,medium,2.0,...,vehicle_java2.jpg,Slovak,2,5,"c++(low), php(medium), delphi(low)",mupliple_choice_rectangle.jpg,131,27978,213.572519,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,215,22,male,Slovak,medium,no,no,Java,medium,1.0,...,vehicle_java2.jpg,slovak,1,2,C (medium) c# (medium) Java (medium),vehicle_java2.jpg,230,63208,274.817391,215
854,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,mupliple_choice_rectangle.jpg,79,22713,287.506329,216
855,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,mupliple_choice_vehicle.jpg,75,13322,177.626667,216
856,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,rectangle_java2.jpg,148,25092,169.540541,216


In [154]:
import pandas as pd


results_ogama_saccades_base = pd.read_csv('data/emip_dataset/ogama_saccades.txt', delimiter='\t', low_memory=False, on_bad_lines='skip', comment='#')

results_ogama_saccades_base['SubjectName'] = results_ogama_saccades_base['SubjectName'].str[1:].astype(int)

results_ogama_saccades_base = results_ogama_saccades_base[['SubjectName', 'Trial Name', 'CountInTrial', 'StartTime', 'Duration', 'Distance', 'Validity']]

results_ogama_saccades_base['Duration'] = results_ogama_saccades_base['Duration'].astype(str)
results_ogama_saccades_base['Duration'] = results_ogama_saccades_base['Duration'].str.replace(',', '.').astype(float)
results_ogama_saccades_base['Distance'] = results_ogama_saccades_base['Distance'].astype(str)
results_ogama_saccades_base['Distance'] = results_ogama_saccades_base['Distance'].str.replace(',', '.').astype(float)


grouped = results_ogama_saccades_base.groupby(['SubjectName', 'Trial Name'])

results_ogama_saccades = grouped.agg({
    'Distance': 'mean',
    'Duration': 'mean',
    'CountInTrial': 'sum'
}).reset_index()

results_ogama_saccades.rename(columns={
    'Distance': 'Average Saccade Distance [px]',
    'Duration': 'Average Saccade Duration [ms]',
    'CountInTrial': 'TotalCountInTrial',
	'Trial Name': 'Task'
}, inplace=True)


# We combine the two dataframes into one by combining the columns where the values of Participant are the same
results_ogama_saccades = pd.merge(participants_metadata, results_ogama_saccades, how='inner', left_on=['id'], right_on=['SubjectName'])

results_ogama_saccades

,id,age,gender,mother_tongue,english_level,visual_aid,makeup,experiment_language,expertise_experiment_language,time_experiment_language,...,stimulus_vehicle,mother_tongue_original,time_experiment_language_original,time_programming_original,other_languages_original,SubjectName,Task,Average Saccade Distance [px],Average Saccade Duration [ms],TotalCountInTrial
0,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_rectangle.jpg,115.073584,113.172043,4371
1,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_vehicle.jpg,152.604348,218.674419,3741
2,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,rectangle_java.jpg,138.137303,402.354167,1176
3,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,vehicle_java.jpg,116.809112,171.880829,18721
4,2,24,male,Slovak,medium,glasses,no,Java,medium,2.0,...,vehicle_java2.jpg,Slovak,2,5,"c++(low), php(medium), delphi(low)",2,mupliple_choice_rectangle.jpg,107.148926,19.292308,8515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,215,22,male,Slovak,medium,no,no,Java,medium,1.0,...,vehicle_java2.jpg,slovak,1,2,C (medium) c# (medium) Java (medium),215,vehicle_java2.jpg,138.203372,19.615721,26335
850,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,mupliple_choice_rectangle.jpg,120.083606,71.423077,3081
851,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,mupliple_choice_vehicle.jpg,121.861234,45.621622,2775
852,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,rectangle_java2.jpg,105.042397,57.394558,10878


In [14]:
data_df = show_results2(results_pygaze)
data_df

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),175.54(44.26),158.38(46.66),175.38(46.10),170.76(44.37),0.000,0.051
1,Fixation Count,180 (134),140 (118),176 (138),174 (117),0.000,0.729


In [12]:
data_df = show_results2(results_ogama)
data_df

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),287.60(79.18),307.50(283.71),293.73(121.96),282.78(79.07),0.002,0.158
1,Fixation Count,134 (99),114 (95),134 (105),132 (91),0.003,0.769


![Results Study 4 2](images/results_study_4_2.png)

In [145]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr

def adapt_results(results):
	# Map the data of expertise_experiment_language and expertise_programming to the following values: none = 0, low = 1, medium = 2, high = 3
	results['expertise_experiment_language'] = results['expertise_experiment_language'].map({'none': 0, 'low': 1, 'medium': 2, 'high': 3, 0: 0, 1: 1, 2: 2, 3: 3})
	results['expertise_programming'] = results['expertise_programming'].map({'none': 0, 'low': 1, 'medium': 2, 'high': 3, 0: 0, 1: 1, 2: 2, 3: 3})

	results.loc[results['time_programming'] == '40.0+', 'time_programming'] = 40.0
	results['time_programming'] = results['time_programming'].astype(float)

	return results


def show_results_tasks(results, results_saccades):
	
	results = adapt_results(results)
	results_saccades = adapt_results(results_saccades)
	
	results_mean_per_subject = results.groupby(['id']).mean(numeric_only=True)
	results_saccades_mean_per_subject = results_saccades.groupby(['id']).mean(numeric_only=True)

	# Vehicle Task performance

	results_vehicle = results[results['stimulus_vehicle'] == results['Task']]
	# Group the results by the id of the participant and calculate the mean of the Fixation Count and Total Fixation Duration
	results_vehicle_mean_per_subject = results_vehicle.groupby(['id']).mean(numeric_only=True)[["correct_vehicle", "Fixation Count", "Total Fixation Duration [ms]"]]

	spearman_vehicle_count = spearmanr(results_vehicle_mean_per_subject['Fixation Count'], results_vehicle_mean_per_subject['correct_vehicle'])

	spearman_vehicle_dur = spearmanr(results_vehicle_mean_per_subject['Total Fixation Duration [ms]'], results_vehicle_mean_per_subject['correct_vehicle'])

	results_saccades_vehicle = results_saccades[results_saccades['stimulus_vehicle'] == results_saccades['Task']]
	# Group the results by the id of the participant and calculate the mean of the Fixation Count and Total Fixation Duration
	results_saccades_vehicle_mean_per_subject = results_saccades_vehicle.groupby(['id']).mean(numeric_only=True)[["correct_vehicle", "Average Saccade Distance [px]", "Average Saccade Duration [ms]"]]

	spearman_saccades_vehicle_dist = spearmanr(results_saccades_vehicle_mean_per_subject['Average Saccade Distance [px]'], results_saccades_vehicle_mean_per_subject['correct_vehicle'])

	spearman_saccades_vehicle_dur = spearmanr(results_saccades_vehicle_mean_per_subject['Average Saccade Duration [ms]'], results_saccades_vehicle_mean_per_subject['correct_vehicle'])





	# Rectangle Task performance

	results_rectangle = results[results['stimulus_rectangle'] == results['Task']]
	# Group the results by the id of the participant and calculate the mean of the Fixation Count and Total Fixation Duration
	results_rectangle_mean_per_subject = results_rectangle.groupby(['id']).mean(numeric_only=True)[["correct_rectangle", "Fixation Count", "Total Fixation Duration [ms]"]]

	spearman_rectangle_count = spearmanr(results_rectangle_mean_per_subject['Fixation Count'], results_rectangle_mean_per_subject['correct_rectangle'])

	spearman_rectangle_dur = spearmanr(results_rectangle_mean_per_subject['Total Fixation Duration [ms]'], results_rectangle_mean_per_subject['correct_rectangle'])

	results_saccades_rectangle = results_saccades[results_saccades['stimulus_rectangle'] == results_saccades['Task']]
	# Group the results by the id of the participant and calculate the mean of the Fixation Count and Total Fixation Duration
	results_saccades_rectangle_mean_per_subject = results_saccades_rectangle.groupby(['id']).mean(numeric_only=True)[["correct_rectangle", "Average Saccade Distance [px]", "Average Saccade Duration [ms]"]]

	spearman_saccades_rectangle_dist = spearmanr(results_saccades_rectangle_mean_per_subject['Average Saccade Distance [px]'], results_saccades_rectangle_mean_per_subject['correct_rectangle'])

	spearman_saccades_rectangle_dur = spearmanr(results_saccades_rectangle_mean_per_subject['Average Saccade Duration [ms]'], results_saccades_rectangle_mean_per_subject['correct_rectangle'])




	# Self Eval Experiment language



	results_self_eval_exp = results_mean_per_subject[['expertise_experiment_language', 'Fixation Count', 'Total Fixation Duration [ms]']]
	spearman_self_eval_exp_count = spearmanr(results_self_eval_exp['Fixation Count'], results_self_eval_exp['expertise_experiment_language'])

	spearman_self_eval_exp_dur = spearmanr(results_self_eval_exp['Total Fixation Duration [ms]'], results_self_eval_exp['expertise_experiment_language'])

	results_saccades_self_eval_exp = results_saccades_mean_per_subject[['expertise_experiment_language', 'Average Saccade Distance [px]', 'Average Saccade Duration [ms]']]

	spearman_saccades_self_eval_exp_dist = spearmanr(results_saccades_self_eval_exp['Average Saccade Distance [px]'], results_self_eval_exp['expertise_experiment_language'])






	# Self Eval Programming language

	results_self_eval_prog = results_mean_per_subject[['expertise_programming', 'Fixation Count', 'Total Fixation Duration [ms]']]
	spearman_self_eval_prog_count = spearmanr(results_self_eval_prog['Fixation Count'], results_self_eval_prog['expertise_programming'])
	spearman_self_eval_prog_dur = spearmanr(results_self_eval_prog['Total Fixation Duration [ms]'], results_self_eval_prog['expertise_programming'])

	results_saccades_self_eval_prog = results_saccades_mean_per_subject[['expertise_programming', 'Average Saccade Distance [px]', 'Average Saccade Duration [ms]']]
	spearman_saccades_self_eval_prog_dist = spearmanr(results_saccades_self_eval_prog['Average Saccade Distance [px]'], results_self_eval_prog['expertise_programming'])
	spearman_saccades_self_eval_prog_dur = spearmanr(results_saccades_self_eval_prog['Average Saccade Duration [ms]'], results_self_eval_prog['expertise_programming'])



	# Time Experiment language

	results_time_exp = results_mean_per_subject[['time_experiment_language', 'Fixation Count', 'Total Fixation Duration [ms]']]
	spearman_time_exp_count = spearmanr(results_time_exp['Fixation Count'], results_time_exp['time_experiment_language'])
	spearman_time_exp_dur = spearmanr(results_time_exp['Total Fixation Duration [ms]'], results_time_exp['time_experiment_language'])

	results_saccades_time_exp = results_saccades_mean_per_subject[['time_experiment_language', 'Average Saccade Distance [px]', 'Average Saccade Duration [ms]']]
	spearman_saccades_time_exp_dist = spearmanr(results_saccades_time_exp['Average Saccade Distance [px]'], results_time_exp['time_experiment_language'])
	spearman_saccades_time_exp_dur = spearmanr(results_saccades_time_exp['Average Saccade Duration [ms]'], results_time_exp['time_experiment_language'])

	# Time Programming language

	results_time_prog = results.groupby(['id']).mean(numeric_only=True)[['time_programming', 'Fixation Count', 'Total Fixation Duration [ms]']]
	spearman_time_prog_count = spearmanr(results_time_prog['Fixation Count'], results_time_prog['time_programming'])
	spearman_time_prog_dur = spearmanr(results_time_prog['Total Fixation Duration [ms]'], results_time_prog['time_programming'])

	results_saccades_time_prog = results_saccades_mean_per_subject[['time_programming', 'Average Saccade Distance [px]', 'Average Saccade Duration [ms]']]
	spearman_saccades_time_prog_dist = spearmanr(results_saccades_time_prog['Average Saccade Distance [px]'], results_time_prog['time_programming'])
	spearman_saccades_time_prog_dur = spearmanr(results_saccades_time_prog['Average Saccade Duration [ms]'], results_time_prog['time_programming'])


	# We create a table to show the results

	data = [
		[
			'Vehicle Task', 
			'%.4f' % spearman_vehicle_count.correlation,
			'%.4f' % spearman_vehicle_dur.correlation,
			'%.4f' % spearman_saccades_vehicle_dist.correlation,
			'%.4f' % spearman_saccades_vehicle_dur.correlation
		],
		[
			'Rectangle Task', 
			'%.4f' % spearman_rectangle_count.correlation,
			'%.4f' % spearman_rectangle_dur.correlation,
			'%.4f' % spearman_saccades_rectangle_dist.correlation,
			'%.4f' % spearman_saccades_rectangle_dur.correlation
		],
		[
			'Self Eval Experiment Language', 
			'%.4f' % spearman_self_eval_exp_count.correlation,
			'%.4f' % spearman_self_eval_exp_dur.correlation,
			'%.4f' % spearman_saccades_self_eval_exp_dist.correlation,
			'%.4f' % spearman_saccades_self_eval_exp_dist.correlation
		],
		[
			'Self Eval Programming Language', 
			'%.4f' % spearman_self_eval_prog_count.correlation,
			'%.4f' % spearman_self_eval_prog_dur.correlation,
			'%.4f' % spearman_saccades_self_eval_prog_dist.correlation,
			'%.4f' % spearman_saccades_self_eval_prog_dur.correlation
		],
		[
			'Time Experiment Language', 
			'%.4f' % spearman_time_exp_count.correlation,
			'%.4f' % spearman_time_exp_dur.correlation,
			'%.4f' % spearman_saccades_time_exp_dist.correlation,
			'%.4f' % spearman_saccades_time_exp_dur.correlation
		],
		[
			'Time Programming Language', 
			'%.4f' % spearman_time_prog_count.correlation,
			'%.4f' % spearman_time_prog_dur.correlation,
			'%.4f' % spearman_saccades_time_prog_dist.correlation,
			'%.4f' % spearman_saccades_time_prog_dur.correlation
		]
	]


	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'Number of Fixations', 'Sum of Fixation Duration', 'Saccade Length', 'Saccade Duration']
	return data_df

	


In [161]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
import numpy

def adapt_results(results):
	# Change the type of the column time_programming to float
	results.loc[results['time_programming'] == '40.0+', 'time_programming'] = 40.0
	results['time_programming'] = results['time_programming'].astype(float)
	# We group the data by time_programming with either between 0.5 and 4 and 4+ and dispose of those with less than 0.5
	results = results[results['time_programming'] >= 0.5]

	# Remove NaN values
	results = results.dropna(subset=['time_programming'])

	# Add a new column time_programming_grouped that groups the time_programming into two groups: 0 = 0.5-4 and 1 = 4+
	results['time_programming_grouped'] = pd.cut(results['time_programming'], [0.5, 4, 40], labels=['0', '1'])
	results = results.dropna(subset=['time_programming_grouped'])
	results['time_programming_grouped'] = results['time_programming_grouped'].astype(int)
	return results



def show_results_expertise(results, results_saccades):
	results = adapt_results(results)
	results_saccades = adapt_results(results_saccades)

	# Add a new column time_programming_grouped that groups the time_programming into two groups: 0 = 0.5-4 and 1 = 4+
	results['time_programming_grouped'] = pd.cut(results['time_programming'], [0.5, 4, 40], labels=['0', '1'])
	results = results.dropna(subset=['time_programming_grouped'])
	results['time_programming_grouped'] = results['time_programming_grouped'].astype(int)


	# Number of fixations
	results_fixations_count_novices = results[results['time_programming_grouped'] == 0].groupby(['id'])['Fixation Count'].sum()
	results_mean_fixations_count_novices = results_fixations_count_novices.mean()
	results_std_fixations_count_novices = results_fixations_count_novices.std()

	results_fixations_count_experts = results[results['time_programming_grouped'] == 1].groupby(['id'])['Fixation Count'].sum()
	results_mean_fixations_count_experts = results_fixations_count_experts.mean()
	results_std_fixations_count_experts = results_fixations_count_experts.std()

	
	
	# Total fixation duration
	results_fixations_dur_novices = results[results['time_programming_grouped'] == 0].groupby(['id'])['Total Fixation Duration [ms]'].sum()
	results_mean_fixations_dur_novices = results_fixations_dur_novices.mean()
	results_std_fixations_dur_novices = results_fixations_dur_novices.std()

	results_fixations_dur_experts = results[results['time_programming_grouped'] == 1].groupby(['id'])['Total Fixation Duration [ms]'].sum()
	results_mean_fixations_dur_experts = results_fixations_dur_experts.mean()
	results_std_fixations_dur_experts = results_fixations_dur_experts.std()



	# Saccade length
	results_saccades_dist_novices = results_saccades[results_saccades['time_programming_grouped'] == 0].groupby(['id'])['Average Saccade Distance [px]'].sum()
	results_mean_saccades_dist_novices = results_saccades_dist_novices.mean()
	results_std_saccades_dist_novices = results_saccades_dist_novices.std()

	results_saccades_dist_experts = results_saccades[results_saccades['time_programming_grouped'] == 1].groupby(['id'])['Average Saccade Distance [px]'].sum()
	results_mean_saccades_dist_experts = results_saccades_dist_experts.mean()
	results_std_saccades_dist_experts = results_saccades_dist_experts.std()


	# Saccade duration
	results_saccades_dur_novices = results_saccades[results_saccades['time_programming_grouped'] == 0].groupby(['id'])['Average Saccade Duration [ms]'].sum()
	results_mean_saccades_dur_novices = results_saccades_dur_novices.mean()
	results_std_saccades_dur_novices = results_saccades_dur_novices.std()

	results_saccades_dur_experts = results_saccades[results_saccades['time_programming_grouped'] == 1].groupby(['id'])['Average Saccade Duration [ms]'].sum()
	results_mean_saccades_dur_experts = results_saccades_dur_experts.mean()
	results_std_saccades_dur_experts = results_saccades_dur_experts.std()





	# We create a table to show the results

	data = [
		[
			'Number of Fixations', 
			'Experts',
			'%.2f' % results_mean_fixations_count_experts,
			'%.2f' % results_std_fixations_count_experts
		],
		[
			'Number of Fixations', 
			'Novices',
			'%.2f' % results_mean_fixations_count_novices,
			'%.2f' % results_std_fixations_count_novices
		],
		[
			'Total Fixation Duration [ms]', 
			'Experts',
			'%.2f' % results_mean_fixations_dur_experts,
			'%.2f' % results_std_fixations_dur_experts
		],
		[
			'Total Fixation Duration [ms]', 
			'Novices',
			'%.2f' % results_mean_fixations_dur_novices,
			'%.2f' % results_std_fixations_dur_novices
		],
		[
			'Saccade Length', 
			'Experts',
			'%.2f' % results_mean_saccades_dist_experts,
			'%.2f' % results_std_saccades_dist_experts
		],
		[
			'Saccade Length', 
			'Novices',
			'%.2f' % results_mean_saccades_dist_novices,
			'%.2f' % results_std_saccades_dist_novices
		],
		[
			'Saccade Duration', 
			'Experts',
			'%.2f' % results_mean_saccades_dur_experts,
			'%.2f' % results_std_saccades_dur_experts
		],
		[
			'Saccade Duration', 
			'Novices',
			'%.2f' % results_mean_saccades_dur_novices,
			'%.2f' % results_std_saccades_dur_novices
		]

	]

	data_df = pd.DataFrame(data)
	data_df.columns = ['Eye Movement Measure', 'Level of expertise', 'Average of actual data', 'SD']
	return data_df


In [173]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
import numpy

def adapt_results(results):
	# Change the type of the column time_programming to float
	results.loc[results['time_programming'] == '40.0+', 'time_programming'] = 40.0
	results['time_programming'] = results['time_programming'].astype(float)
	# We group the data by time_programming with either between 0.5 and 4 and 4+ and dispose of those with less than 0.5
	results = results[results['time_programming'] >= 0.5]

	# Remove NaN values
	results = results.dropna(subset=['time_programming'])

	# Add a new column time_programming_grouped that groups the time_programming into two groups: 0 = 0.5-4 and 1 = 4+
	results['time_programming_grouped'] = pd.cut(results['time_programming'], [0.5, 4, 40], labels=['0', '1'])
	results = results.dropna(subset=['time_programming_grouped'])
	results['time_programming_grouped'] = results['time_programming_grouped'].astype(int)
	return results

from numpy import var, mean, sqrt
from pandas import Series

def cohend(d1: Series, d2: Series) -> float:

    # calculate the size of samples
    n1, n2 = len(d1), len(d2)

    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)

    # calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))

    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)

    # return the effect size
    return (u1 - u2) / s

def mannwhitney_z(U, nx, ny):
	return (U - (nx * ny / 2)) / sqrt(nx * ny * (nx + ny + 1) / 12)


def get_results(x, y):
	nx = len(x)
	ny = len(y)
	U1, p = mannwhitneyu(x, y)
	U2 = nx * ny - U1
	U = min(U1, U2)
	z = mannwhitney_z(U, nx, ny)
	cohend_value = cohend(x, y)
	return U, p, z, cohend_value


def show_results_expertise_significance(results, results_saccades):
	results = adapt_results(results)
	results_saccades = adapt_results(results_saccades)


	mannwhitneyu_fixations_count = get_results(results[results['time_programming_grouped'] == 0]['Fixation Count'], results[results['time_programming_grouped'] == 1]['Fixation Count'])

	mannwhitneyu_fixations_dur = get_results(results[results['time_programming_grouped'] == 0]['Total Fixation Duration [ms]'], results[results['time_programming_grouped'] == 1]['Total Fixation Duration [ms]'])

	mannwhitneyu_saccades_dist = get_results(results_saccades[results_saccades['time_programming_grouped'] == 0]['Average Saccade Distance [px]'], results_saccades[results_saccades['time_programming_grouped'] == 1]['Average Saccade Distance [px]'])

	mannwhitneyu_saccades_dur = get_results(results_saccades[results_saccades['time_programming_grouped'] == 0]['Average Saccade Duration [ms]'], results_saccades[results_saccades['time_programming_grouped'] == 1]['Average Saccade Duration [ms]'])

	# We create a table to show the results
	data = [
		[
			'Number of Fixations', 
			'%.4f' % mannwhitneyu_fixations_count[0],
			'%.4f' % mannwhitneyu_fixations_count[1],
			'%.4f' % mannwhitneyu_fixations_count[2],
			'%.4f' % mannwhitneyu_fixations_count[3]
		],
		[
			'Total Fixation Duration [ms]', 
			'%.4f' % mannwhitneyu_fixations_dur[0],
			'%.4f' % mannwhitneyu_fixations_dur[1],
			'%.4f' % mannwhitneyu_fixations_dur[2],
			'%.4f' % mannwhitneyu_fixations_dur[3]
		],
		[
			'Saccade Length', 
			'%.4f' % mannwhitneyu_saccades_dist[0],
			'%.4f' % mannwhitneyu_saccades_dist[1],
			'%.4f' % mannwhitneyu_saccades_dist[2],
			'%.4f' % mannwhitneyu_saccades_dist[3]
		],
		[
			'Saccade Duration', 
			'%.4f' % mannwhitneyu_saccades_dur[0],
			'%.4f' % mannwhitneyu_saccades_dur[1],
			'%.4f' % mannwhitneyu_saccades_dur[2],
			'%.4f' % mannwhitneyu_saccades_dur[3]
		]
	]

	data_df = pd.DataFrame(data)
	data_df.columns = ['Eye Movement Feature', 'U', 'p', 'Z', 'Cohen\'s D']
	return data_df



In [146]:
show_results_tasks(results_pygaze, results_pygaze_saccades)

,,Number of Fixations,Sum of Fixation Duration,Saccade Length,Saccade Duration
0,Vehicle Task,-0.0195,-0.0090,0.0151,0.0063
1,Rectangle Task,0.0890,0.1460,-0.0289,-0.0237
2,Self Eval Experiment Language,0.0817,0.0984,-0.0188,-0.0188
3,Self Eval Programming Language,0.0493,0.0543,-0.0155,-0.0101
4,Time Experiment Language,0.0088,0.0462,-0.0870,-0.0637
5,Time Programming Language,-0.0330,0.0221,-0.1381,-0.1237


In [158]:
show_results_tasks(results_ogama, results_ogama_saccades)

,,Number of Fixations,Sum of Fixation Duration,Saccade Length,Saccade Duration
0,Vehicle Task,-0.0090,-0.0188,0.0280,0.0124
1,Rectangle Task,0.0860,0.1278,0.0642,-0.0339
2,Self Eval Experiment Language,0.0732,0.0909,0.1295,0.1295
3,Self Eval Programming Language,0.0304,0.0484,-0.0002,-0.0035
4,Time Experiment Language,0.0045,0.0291,0.1163,-0.0017
5,Time Programming Language,-0.0630,-0.0285,-0.0223,-0.0469


![Results Study EMIP](images/results_study_emip.png)

In [163]:
show_results_expertise(results_pygaze, results_pygaze_saccades)

,Eye Movement Measure,Level of expertise,Average of actual data,SD
0,Number of Fixations,Experts,677.90,303.07
1,Number of Fixations,Novices,737.74,310.88
2,Total Fixation Duration [ms],Experts,122433.95,59548.17
3,Total Fixation Duration [ms],Novices,125388.28,55467.08
4,Saccade Length,Experts,496.59,212.57
5,Saccade Length,Novices,553.04,172.59
6,Saccade Duration,Experts,5179.34,10279.73
7,Saccade Duration,Novices,5202.96,6778.71


In [164]:
show_results_expertise(results_ogama, results_ogama_saccades)

,Eye Movement Measure,Level of expertise,Average of actual data,SD
0,Number of Fixations,Experts,495.94,208.42
1,Number of Fixations,Novices,562.25,233.16
2,Total Fixation Duration [ms],Experts,140738.60,61862.86
3,Total Fixation Duration [ms],Novices,150316.54,58356.25
4,Saccade Length,Experts,476.43,82.88
5,Saccade Length,Novices,497.14,112.42
6,Saccade Duration,Experts,345.23,773.16
7,Saccade Duration,Novices,396.48,770.95


![Results Study EMIP 2](images/results_study_emip_2.png)

In [174]:
show_results_expertise_significance(results_pygaze, results_pygaze_saccades)

,Eye Movement Feature,U,p,Z,Cohen's D
0,Number of Fixations,71641.5000,0.3163,-1.0023,0.1134
1,Total Fixation Duration [ms],74406.0000,0.9095,-0.1138,0.0314
2,Saccade Length,61524.0000,0.0000,-4.2539,0.2125
3,Saccade Duration,63148.0000,0.0002,-3.7320,0.0020


In [175]:
show_results_expertise_significance(results_ogama, results_ogama_saccades)

,Eye Movement Feature,U,p,Z,Cohen's D
0,Number of Fixations,68609.5000,0.0897,-1.6969,0.1775
1,Total Fixation Duration [ms],70404.5000,0.2651,-1.1146,0.1013
2,Saccade Length,66573.0000,0.0287,-2.1879,0.1645
3,Saccade Duration,66344.0000,0.0237,-2.2626,0.0492


![Results Study EMIP 3](images/results_study_emip_3.png)